# BioExplorer - Rat Non-barrel Somatosensory Cortex Anatomy
![](../../bioexplorer_neurons_banner.png)

Source: https://zenodo.org/record/6906785#.YwzhpdVBxH7

In [ ]:
import os
import bluepysnap  # https://bluebrainsnap.readthedocs.io/en/stable/
import neurom  # https://neurom.readthedocs.io/en/stable/
import pandas
import numpy
import scipy

from neurom import viewer

data_folder = os.getenv('O1_DATA_ANATOMY')
fn_extr = os.path.join(data_folder, 'circuit_config.json')
circ_extr = bluepysnap.Circuit(fn_extr)

population_name = 'S1nonbarrel_neurons'


### Neuron populations and their properties

The model contains a number of neuron (or node) populations. Let's list them, then I'll explain them

In [ ]:
print(circ_extr.nodes.population_names)

We find five populations. One population is the most important one: 'S1nonbarrel_neurons' denotes the neurons actually in the model. The ones that can be simulated. The other four denote external populations that innervate the 'S1nonbarrel_neurons':

'POm' and 'VPM' are innervating neurons from those thalamic nuclei respectively. Representing a "matrix-type" and "core-type" projection respectively.
'external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical' and 'external_midrange__S1nonbarrel_neurons__chemical' denote nonbarrel S1 neurons outside the modeled population that innervate the modeled population. The difference between them is as follows: 'external_midrange...' innervates via long-range connections, while 'external_S1...' innervates via local connections. For an explanation of "local" vs. "long-range" connections, see the accompanying manuscript. 


Now we can load the properties of the neurons in these populations.

In [ ]:
lst_properties = ['layer', 'region', 'synapse_class', 'population',
                  'orientation_x', 'orientation_y', 'orientation_z', 'orientation_w',
                  'morph_class', 'model_type', 'mtype', 'morphology',
                  'x', 'y', 'z']
n_props = circ_extr.nodes.get(properties=lst_properties)
display(n_props.loc[population_name])

'layer', 'region' specify in which cortical region and layer a neuron was placed.

'x', 'y', 'z' specify the exact coordinate (in um) in the atlas that a neuron was placed at.

'orientation_x', 'orientation_y', 'orientation_z', 'orientation_w' define a quarternion that specifies the rotation that is applied to the neuron morphology in placement.

'synapse_class', 'morph_class', 'mtype' specify the class of neuron in terms of being excitatory vs inhibitory, pyramidal vs interneuron and the morphological type.

'morphology' specifies the name of the morphological reconstruction of the neuron

'model_type' specifies whether a neuron is actually part of the model that can be simulated ("biophysical") or an external innervator ("virtual"). As explained above, all 'S1nonbarrel_neurons' are "biophysical", the others are all "virtual".


Note that for the "virtual" neurons some properties are unassigned and therefore reported as NaN. 

In [ ]:
display(n_props.loc['external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical'])  # Some columns are NaN

### Neuron morphologies

For the 'S1nonbarrel_neurons' population, we can then load their morphologies with some simple helper functions.

For this example, we load and draw and exemplary neuron. For more that can be done with the loaded morphology, see the NeuroM documentation linked in the first cell of this notebook.

In [ ]:
S1nonbarrel_cfg = circ_extr.config["networks"]["nodes"][0]["populations"][population_name]

def transform_neuron(nrn_morph, neuron_row):
    rot = scipy.spatial.transform.Rotation.from_quat(neuron_row[["orientation_x", "orientation_y",
                                                                 "orientation_z", "orientation_w"]].values)
    rot = neurom.geom.transform.Rotation(rot.as_matrix())
    tl = neurom.geom.transform.Translation(neuron_row[["x", "y", "z"]].values)
    return nrn_morph.transform(rot).transform(tl)
    
def load_neuron(population_cfg, neuron_row, transform=True):
    assert neuron_row["model_type"] == "biophysical", "Cannot load morphologies of virtual neurons"
    fn = os.path.join(
        population_cfg["alternate_morphologies"]["neurolucida-asc"],
        neuron_row["morphology"] + ".asc"
    )
    nrn = neurom.load_neuron(fn)
    if transform:
        nrn = transform_neuron(nrn, neuron_row)
    return nrn

nrn_morph = load_neuron(S1nonbarrel_cfg, n_props.loc[(population_name, 0)])
viewer.draw(nrn_morph)

### Synaptic connectivity

The synaptic connectivity of the model is accessed using circ_extr.edges.
As an example, let's find out which neurons innervate an exemplary neuron

In [ ]:
# The method we use below requires node ids as input. So we load the ids of the "S1nonbarrel_neurons"
node_ids = circ_extr.nodes.ids().filter_population(population_name)

# This will be our examplary neuron. Let's look at its properties
display(n_props.loc[node_ids[0]])

# Which neurons are afferent to the first one?
aff = circ_extr.edges.afferent_nodes(node_ids[0])
# What's returned can be used as index into the neuron properties data frame
aff_props = n_props.loc[aff]

# Look at the regions of neurons innervating the exemplary one from within the model population
display(aff_props["region"][population_name].value_counts())

We see that the exemplary neuron is a L1_DAC neuron in the S1FL (front limb) area. From within the modeled population, it is mostly innervated by other S1FL neurons.

Let's see what external neurons innervate it.

In [ ]:
# External innervation via local connections
display(aff_props["region"]["external_S1nonbarrel_neurons__S1nonbarrel_neurons__chemical"].value_counts())

# External innervation via long-range connections
display(aff_props["region"]["external_midrange__S1nonbarrel_neurons__chemical"].value_counts())


Unsurprisingly, we see that external innervation via local connection comes also mostly from S1FL. For long-range connections though, S1ULp is dominating.

## Visualize with the Blue Brain BioExplorer

### Connect to back-end

In [ ]:
from bioexplorer import BioExplorer

url = 'localhost:5000'
be = BioExplorer(url)

core = be.core_api()
status = be.reset_scene()

### Load neurons

In [ ]:
assembly_name = 'Neurons'
be.remove_assembly(assembly_name)
neurons_assembly = be.add_assembly(assembly_name)

neurons_model = be.add_neurons(
    assembly_name=assembly_name,
    population_name=population_name,
    morphology_representation=be.MORPHOLOGY_REPRESENTATION_SEGMENT,
    morphology_color_scheme=be.MORPHOLOGY_COLOR_SCHEME_SECTION_TYPE,
    use_sdf=True,
    sql_node_filter='guid%100=0' # Only import 1 neuron in 100
)

be.reset_camera()